# Configuration

In [ ]:
# Pre-requisite Configuration
# ------------------------------------------------------------------------------ #
# For testing
# ------------------------------------------------------------------------------ #

%load_ext autoreload
%autoreload 2

import os 
from modules.util.config import get_config_by_id, get_config_global
from modules.util.database import SQLAlchemyClient
from modules.util.helpers import Logger


# ------------------------------------------------------------------------------ #
# Configuration
# ------------------------------------------------------------------------------ #

CONFIG_ID = 'CUSTOM_TEST'
TRANSFORM = get_config_by_id(CONFIG_ID)["load"]["indicator"]
EXTRACTION_DIR = TRANSFORM["directory"]
REPORTS_DIR = f"{EXTRACTION_DIR}/reports"
CONFIG_GLOBAL = get_config_global().get('indicators').get('transform')

# ------------------------------------------------------------------------------ #
# Create directories
# ------------------------------------------------------------------------------ #
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)

# ------------------------------------------------------------------------------ #
# Logger
# ------------------------------------------------------------------------------ #

log = Logger.get_logger(CONFIG_ID)

Logger.blank_line(log)
log.info("** LOAD - INDICATORS **")
Logger.blank_line(log)

log.info(f"Extraction Directory: {EXTRACTION_DIR}")
log.info(f"Reports Directory: {REPORTS_DIR}")

db = SQLAlchemyClient(CONFIG_ID)

# Validate Characteristics in APM

This step validates if all characteristics are synced from ERP and are present in APM

In [ ]:
# Validate ERP Characteristics in APM
# ------------------------------------------------------------------------------ #
# this step ensures that the characteristics are migrated / created from ERP to APM

# standard imports
import pandas as pd

# custom imports
from modules.util.database import V_ERPCharacteristics
from modules.apm.manage_indicators import ApiCharacteristics
from concurrent.futures import ThreadPoolExecutor, as_completed
from modules.util.api import APIException

file_err = rf"{REPORTS_DIR}/3_Characteristics_Validate_ERR.csv"

chars = db.select(
    V_ERPCharacteristics,
    fields=['ERPCharacteristic','CharcInternalID']
)

df_chars = pd.DataFrame(chars)

api = ApiCharacteristics(CONFIG_ID)
error = []

def call_api(id):
    return(api.search_characteristic(id))

with ThreadPoolExecutor(max_workers=20) as executor:
    future_char = {executor.submit(call_api, char): char for char in df_chars['CharcInternalID']}

    for future in as_completed(future_char):
        char = future_char[future]
        try:
            data = future.result()
            if data:
                continue
        except APIException as api_e:
            if api_e.status_code == 404:
                erp_char = df_chars.loc[df_chars['CharcInternalID'] == char, 'ERPCharacteristic'].values[0]
                message = f"Characteristic {char} ({erp_char}) not found in APM"
                api_e.response = message
                log.error(message)
            else:
                log.error(f"API Exception: {char} - {api_e.status_code} - {api_e.response} - {api_e.endpoint}")
            
            err = {
                'characteristic': char,
                'status_code': api_e.status_code,
                'response': api_e.response,
                'endpoint': api_e.endpoint
            }
            error.append(err)

if error:
    log.error(f"{len(error)} characteristics had errors.")
    df_err = pd.DataFrame(error)
    df_err.to_csv(file_err,index=False)
    log.error(f"{file_err} generated.")
else:
    log.info("ERP Characteristics are present in APM.")